# Define Your Model

## Overview

In this notebook we will introduce 3 basic components for Renormalizer: `Op`, `BasisSet` and `Model`.
These components are essential for MPS/MPO construction/manipulation for a given physical model.

## `Op`

`Op` is the abbreviation for "operators". It offers a convenient interface to represent operators of your interest.
`Op` is the bridge between symbolic math equations and numerical tensors in MPS/MPO.

In [1]:
from renormalizer import Op

2025-12-19 12:48:31,540[INFO] Use NumPy as backend


2025-12-19 12:48:31,541[INFO] numpy random seed is 9012


2025-12-19 12:48:31,542[INFO] random seed is 1092


2025-12-19 12:48:31,551[INFO] Git Commit Hash: 4473613f230a9bb73f01438592a7fbb8f73bece1


2025-12-19 12:48:31,552[INFO] use 64 bits


### Basics

There are 3 basic attributes for an operator:

- The operation or its symbol, such as $\hat X$, $\hat a^\dagger$, $\hat p^2$
- The DOF at wich the operation is operated on, such as "spin 0", "exiton 1". In mathematical expression this is usually the subscript for an operator, i.e., $\hat X_0$ or $\hat a^\dagger_1$
- The factor or coefficient for the operator.

There is an additional useful attributes that is sometimes quite useful: the quantum number. The idea of quantum number will be described in detail in another tutorial.

`Op` are constructed from the above 4 attributes. The first 2 are necessary and the last 2 are optional.

In [2]:
Op("X", 0, factor=0.5, qn=0)

Op('X', [0], 0.5)

In renormalizer, you may use anything that can be hashed and compared to denote a DOF.
Common examples include `int`, `str`, and `tuple` of them.

In [3]:
Op("X", ("spin 0", "cool stuff", 2077))

Op('X', [('spin 0', 'cool stuff', 2077)], 1.0)

You may wonder what are the allowed symbols for operators. For that please refer to the `BasisSet` section.

Products of operators can be constructed intuitively

In [4]:
Op(r"a^\dagger a", [0, 1])

Op('a^\\dagger a', [0, 1], 1.0, [[1], [-1]])

Here, DOFs for the operators are specified through a list, where the first element is the DOF for $a^\dagger$ and the second element is the DOF for $a$.

Note that using tuple to specify the DOF has a totally different meaning. Renormalizer will recognize the tuple as a single DOF and set all DOFs in the operator to that DOF. 

In [5]:
Op(r"a^\dagger a", (0, 1))

Op('a^\\dagger a', [(0, 1), (0, 1)], 1.0, [[1], [-1]])

Operator symbols for different DOFs should be separated by a space

In [6]:
Op("X X X", [0, 1, 2])  # note that Op("XXX", [0, 1, 2]) does not work

Op('X X X', [0, 1, 2], 1.0)

`Op` also has a lot of useful functions and attributes, please refer to the API document for more details.

### Operations and `OpSum`

Common and simple operations between operators are supported

In [7]:
op1 = Op("X", 0, 0.5)
op2 = Op("Z", 1, 0.5)
op1 * op2

Op('X Z', [0, 1], 0.25)

Addition between `Op` will result in an `OpSum` instance that is a subclass of list.

In [8]:
op1 + op2, type(op1 + op2)

([Op('X', [0], 0.5), Op('Z', [1], 0.5)], renormalizer.model.op.OpSum)

`OpSum` supports simple operator algebra such as multiplication/addition, etc.

In [9]:
# multiplication
opsum = op1 + op2
opsum * op1

[Op('X X', [0, 0], 0.25), Op('Z X', [1, 0], 0.25)]

In [10]:
10 * op1

Op('X', [0], 5.0)

In [11]:
opsum * opsum

[Op('X X', [0, 0], 0.25),
 Op('X Z', [0, 1], 0.25),
 Op('Z X', [1, 0], 0.25),
 Op('Z Z', [1, 1], 0.25)]

In [12]:
# addition/subtraction
opsum -= op2
opsum

[Op('X', [0], 0.5), Op('Z', [1], 0.5), Op('Z', [1], -0.5)]

In [13]:
opsum.simplify()

[Op('X', [0], 0.5)]

However, in general the functionalities are limited and the performance is not optimized.
We recommand using [SymPy](https://www.sympy.org/en/index.html) for advanced symbolic mathematics and convert the final result to Renormalizer `Op`.

## `BasisSet`
An essential step for converting symbolic operators into numeric tensors is to specify a set of basis.
Renormalizer includes a zoo of basis set with an emphasis for electron-phonon or vibronic models.

In [14]:
import renormalizer
[s for s in dir(renormalizer) if s.startswith("Basis")]

['BasisDummy',
 'BasisHalfSpin',
 'BasisHopsBoson',
 'BasisMultiElectron',
 'BasisMultiElectronVac',
 'BasisSHO',
 'BasisSimpleElectron',
 'BasisSineDVR']

Each BasisSet is associated with one or more DOFs. For example, we can setup the spin-1/2 basis set for a spin DOF denoted as `("spin", 0)`

In [15]:
from renormalizer import BasisHalfSpin
b = BasisHalfSpin(("spin", 0))
b

BasisHalfSpin(dof: ('spin', 0), nbas: 2)

Each basis set supports a variety of operations. The spin-1/2 basis set naturally supports Pauli operators such as `"Y"`, `"sigma_z"`.

In [16]:
b.op_mat("Y"), b.op_mat("sigma_z")

(array([[0.+0.j, 0.-1.j],
        [0.+1.j, 0.+0.j]]),
 array([[ 1.,  0.],
        [ 0., -1.]]))

> In Renormalizer, we use `[1, 0]` for spin up and `[0, 1]` for spin down.

For phonon DOF, it is necessary to specify the quanta truncated in the basis set

In [17]:
# SHO represent simple harmonic oscillator
from renormalizer import BasisSHO
# omega is the vibration frequency, nbas is the number of basis
b = BasisSHO(dof=0, omega=50, nbas=4)
# note the different value because of omega
b.op_mat(r"b^\dagger+b"), b.op_mat("x")

(array([[0.        , 1.        , 0.        , 0.        ],
        [1.        , 0.        , 1.41421356, 0.        ],
        [0.        , 1.41421356, 0.        , 1.73205081],
        [0.        , 0.        , 1.73205081, 0.        ]]),
 array([[0.        , 0.1       , 0.        , 0.        ],
        [0.1       , 0.        , 0.14142136, 0.        ],
        [0.        , 0.14142136, 0.        , 0.17320508],
        [0.        , 0.        , 0.17320508, 0.        ]]))

Note that `BasisSHO` supports both first and second-quantized operators such as
$$
\hat x = \sqrt{\frac{1}{2\omega}} (b^\dagger + b)
$$

here $x$ is the mass-weighted coordinate.

The number of possible basis sets is infinite and highly customized basis set for a particular problem is rairly common.
You may subclass the `BasisSet` parent class in `renormalizer.model.basis.BasisSet` to customize the basis set and the numerical behavior of the operators.

Renormalizer supports DVR basis (a.k.a. grid basis) where the basis functions are eigenfunctions of the position operator.
Use the `BasisSineDVR` class or set `dvr=True` in BasisSHO for DVR basis set.

## `Model`

A `Model` is basically made up with a list of `BasisSet` and the Hamiltonian in `OpSum`/`list` form.
A `Model` instance is necessary for any MPS/MPO construction.

In [18]:
from renormalizer import Model
model = Model([BasisHalfSpin(0)], [Op("X", 0)])

For example, the corresponding MPO can be constructed from the above model conveniently

In [19]:
from renormalizer import Mpo
mpo = Mpo(model)
mpo

2025-12-19 12:48:31,697[DEBUG] # of operator terms: 1


2025-12-19 12:48:31,697[DEBUG] Input operator terms: 1


2025-12-19 12:48:31,699[DEBUG] After combination of the same terms: 1


<class 'renormalizer.mps.mpo.Mpo'> with 1 sites

In [20]:
mpo.todense()

array([[0., 1.],
       [1., 0.]])

You may wonder 

- What are the limitations for the model, in particular the operator terms, to construct MPO? For example, is nearest-neighbour interaction or at most two-body operator enforced?
- If any numerical compression is involved in the construction. If so, how to control the accuracy? 
- What are the time cost of constructing MPO for an arbitrary model?

The answer is: renormalizer offers a unique algorithm for exact and efficient MPO construction of arbitrary operators.
It means

- There's **NO limitation** for the model. Long-range interaction, three-body operators, anything you like.
- Numerical compression is not involved and the construction is **exact**.
- The construction time cost is neglegible for most models

Additionally, we also guarantee that the constructed MPO is **optimal** in terms of bond dimension. 

For details of our algorithm, see [A general automatic method for optimal construction of matrix product operators using bipartite graph theory](https://aip.scitation.org/doi/10.1063/5.0018149).

Renormalizer also offers a set of built-in models for fast construction of common models. Please refer to the API document for more details.

In [21]:
import renormalizer
[s for s in dir(renormalizer) if s.endswith("Model")]

['HolsteinModel', 'Model', 'SpinBosonModel', 'TI1DModel']

## Define Custom `BasisSet`
It is quite common to require specific basis set for the problem at hand. The following is an example of defining a custom basis set.

The key to the implementation is to implement the `op_mat` method, which outputs the matrix representation of a local operator.
You should implement the necessary operators here for your model definition.
The input to the function could be either `str` or `Op`. Use `op.symbol` to access the symbol string.

In [22]:
from typing import Union, List
import numpy as np
from renormalizer.model.basis import BasisSet

class CustomTwoLevelSystem(BasisSet):
    """
    A custom two-level system basis set example.
    Implements a simple two-level system with custom operators.
    """
    
    def __init__(self, dof, energy_splitting=1.0, sigmaqn: List[int] = None):
        # Define the basis states and their quantum numbers
        # For a two-level system, we have two states: ground and excited
        if sigmaqn is None:
            sigmaqn = [0, 1]  # Quantum numbers for each state
        
        # Initialize the BasisSet class with the number of states and quantum numbers
        # Must provide `sigma_qn` even if it's all zero.
        super().__init__(dof, nbas=2, sigmaqn=sigmaqn)
        self.energy_splitting = energy_splitting
        
    def op_mat(self, op: Union[Op, str]):
        """
        Define operator matrices for supported symbols:
        - 'sigma_z': Pauli Z matrix
        - 'sigma_x': Pauli X matrix
        - 'n': number operator (|1><1|)
        - 'H_sys': system Hamiltonian (energy_splitting * sigma_z)
        """
        # Convert symbol to string if necessary
        if not isinstance(op, Op):
            op = Op(op, None)
        
        op_symbol, op_factor = op.symbol, op.factor
        if op_symbol == "sigma_z":
            mat = np.array([[1, 0], [0, -1]])
        elif op_symbol == "sigma_x":
            mat = np.array([[0, 1], [1, 0]])
        elif op_symbol == "n":
            mat = np.array([[0, 0], [0, 1]])  # |1><1|
        elif op_symbol == "H_sys":
            # System Hamiltonian: energy_splitting * sigma_z
            mat = self.energy_splitting * np.array([[1, 0], [0, -1]])
        elif op_symbol == "I":
            # always define the identity operator
            mat = np.eye(self.nbas)
        else:
            raise ValueError(f"Unknown operator symbol: {op_symbol}")
        
        return mat * op_factor

    def copy(self, new_dof):
        """Create a copy with a new DOF label"""
        return self.__class__(new_dof, self.energy_splitting, self.sigmaqn)

In [23]:
# Create a model with our custom basis
custom_basis = CustomTwoLevelSystem("qubit", energy_splitting=2.0)
model = Model([custom_basis], [Op("H_sys", "qubit")])

# Create MPOs using our custom symbols
sigma_z_mpo = Mpo(model, Op("sigma_z", "qubit"))
print(sigma_z_mpo.todense())
sigma_x_mpo = Mpo(model, Op("sigma_x", "qubit"))
print(sigma_x_mpo.todense())

2025-12-19 12:48:31,729[DEBUG] # of operator terms: 1


2025-12-19 12:48:31,729[DEBUG] Input operator terms: 1


2025-12-19 12:48:31,731[DEBUG] After combination of the same terms: 1


2025-12-19 12:48:31,732[DEBUG] # of operator terms: 1


2025-12-19 12:48:31,732[DEBUG] Input operator terms: 1


2025-12-19 12:48:31,734[DEBUG] After combination of the same terms: 1


[[ 1.  0.]
 [ 0. -1.]]
[[0. 1.]
 [1. 0.]]


> To improve computational efficiency, when defining the basis set, float numbers are preferred over complex numbers. Complex operators wil usually be finally converted to real operators, such as $H=YY$ or $H=-iY$.

## Units in Renormalizer
All internal numbers in renormalizer are assumed to have the unit of atomic unit (a.u.).
Consequently, all external interfaces such as `factor` in `Op` are assumed to have the unit of atomic unit.

To facilitate unit conversion, You may use the `Quantity` class in Renormalizer to convert the values in other units to atomic units.

In [24]:
from renormalizer import Quantity

In [25]:
v = Quantity(27212, "mev")
v.as_au()

1.0000225550439237

Note that directly using a `Quantity` instance for numerical arguments of model construction will result in errors. 
For example, you should write `BasisSHO(dof=0, omega=Quantity(50).as_au(), nbas=4)` instead of `BasisSHO(dof=0, omega=Quantity(50), nbas=4)`.
The design enforces the user to pay special attention to the units as well as the value of the parameters, which helps to prevent unit/parameter typo.

To convert the unit from a.u. to other supported units, use `as_unit` to convert to the desired unit, and then access the `value` attribute for the value.

In [26]:
Quantity(1, "au").as_unit("mev").value

27211.386245988